# LO12 P25
## Analyse des habitudes d'achat Steam avec l'algorithme Apriori

### LE PELTIER Swan

Ce notebook accompagne un projet d'analyse de données Steam, utilisant l'algorithme Apriori pour extraire des règles d'association et générer des recommandations de jeux. Nous passerons par les étapes suivantes :

1. **Importation des bibliothèques**
2. **Téléchargement et chargement des données depuis KaggleHub**
3. **Prétraitement et nettoyage**
4. **Construction des transactions**
5. **Extraction des itemsets fréquents et génération des règles**
6. **Génération de règles au format CLIPS**
7. **Test interactif des règles CLIPS**
8. **Conclusion**


## Prerequis
Assurez-vous d'avoir installé les bibliothèques nécessaires :
```bash
pip install pandas mlxtend clipspy kagglehub
```


## 1. Import des bibliothèques

In [8]:
import os
import re
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import kagglehub  # interface personnalisée pour Kaggle
import clips      # API Python pour CLIPS

---

## 2. Télécharger et charger les données Steam

Dans cette section, nous téléchargeons le jeu de données [@tamber/steam-video-games](https://www.kaggle.com/datasets/tamber/steam-video-games) depuis KaggleHub et le chargeons dans un DataFrame pandas. Nous vérifions également l'intégrité du fichier car j'ai eu plusieurs problème de donné corompu au moment du téléchargement.

In [9]:
def load_steam_data(dataset_name: str = "tamber/steam-video-games",
                    file_name: str = "steam-200k.csv") -> pd.DataFrame:
    # Téléchargement via KaggleHub
    path = kagglehub.dataset_download(dataset_name)
    print(f"Dataset téléchargé dans : {path}")

    # Construction du chemin vers le fichier et vérification
    file_path = os.path.join(path, file_name)
    if not os.path.exists(file_path) or os.path.getsize(file_path) < 1000:
        raise FileNotFoundError(f"Fichier introuvable ou corrompu : {file_path}")

    # Lecture du CSV sans en-tête, avec noms de colonnes personnalisés
    col_names = ['user_id', 'game_name', 'purchase', 'playtime', 'unused']
    df = pd.read_csv(file_path,
                     encoding='latin-1',
                     delimiter=',',
                     header=None,
                     names=col_names)

    print("Aperçu des données Steam (5 premières lignes) :")
    display(df.head())
    return df

# Chargement des données
steam_df = load_steam_data()
print(f"Nombre total de lignes initiales : {len(steam_df)}")

Dataset téléchargé dans : C:\Users\swanl\.cache\kagglehub\datasets\tamber\steam-video-games\versions\3
Aperçu des données Steam (5 premières lignes) :


,user_id,game_name,purchase,playtime,unused
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


Nombre total de lignes initiales : 200000


La strucutre est faite de sorte a utilisé une ligne par jeux que possède l'utilisateur.

Nous pouvons voir ici que les 5 premières entrée concerne l'utilisateur `151603712` et les différents jeux qu'il possède.

---

## 3. Prétraitement et nettoyage des données

Nous conservons uniquement les colonnes `user_id` et `game_name`, supprimons les doublons et retirons la colonne inutilisée.

In [10]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    # On garde uniquement les colonnes d'intérêt et on supprime les doublons
    df = df[['user_id', 'game_name']]
    df = df.drop_duplicates()
    return df

# Application du prétraitement
clean_df = preprocess_data(steam_df)
print(f"Nombre de lignes après nettoyage : {len(clean_df)}")
display(clean_df.head())

Nombre de lignes après nettoyage : 128804


,user_id,game_name
0,151603712,The Elder Scrolls V Skyrim
2,151603712,Fallout 4
4,151603712,Spore
6,151603712,Fallout New Vegas
8,151603712,Left 4 Dead 2


Cela nous permet de nettoyer les doublons former par la notion (play / purchase) que nous n'intégrons pas dans notre algoritmhe.
Cela nous permet également de travailler avec seulement les données qui nous intérèsse.

---

## 4. Construction des transactions pour l'algorithme Apriori

Nous groupons les jeux par utilisateur pour obtenir une liste de transactions, où chaque transaction représente l'ensemble des jeux d'un utilisateur.

In [11]:
def build_transactions(df: pd.DataFrame) -> list:
    grouped = df.groupby('user_id')['game_name'].apply(list)
    return grouped.tolist()

transactions = build_transactions(clean_df)
print(f"Nombre de transactions : {len(transactions)}")

Nombre de transactions : 12393


---

## 5. Extraction des itemsets fréquents et génération des règles d'association

Nous convertissons les transactions en matrice binaire, appliquons l'algorithme Apriori pour extraire les itemsets fréquents, puis générons les règles d'association avec un seuil de confiance minimal.

Nous utilisons une valeur de support à 0.02 pour intégrer les jeux "niche" pour plus de recommandation.

In [12]:
def mine_association_rules(transactions: list,
                           min_support: float = 0.02,
                           min_confidence: float = 0.5) -> pd.DataFrame:
    # Encodage des transactions au format binaire
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    trans_df = pd.DataFrame(te_ary, columns=te.columns_)

    # Extraction des itemsets fréquents
    frequent_itemsets = apriori(trans_df,
                                min_support=min_support,
                                use_colnames=True)

    # Génération des règles d'association
    rules = association_rules(frequent_itemsets,
                               metric="confidence",
                               min_threshold=min_confidence)
    return rules

# Extraction des règles avec support >= 2% et confiance >= 60%
rules = mine_association_rules(transactions,
                               min_support=0.02,
                               min_confidence=0.6)
print(f"Nombre de règles générées : {len(rules)}")
display(rules.head())

Nombre de règles générées : 5797


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(Arma 2 Operation Arrowhead),(Arma 2 Operation Arrowhead Beta (Obsolete)),0.020173,0.020173,0.020173,1.0,49.572000,1.0,0.019766,inf,1.0,1.0,1.0,1.0
1,(Arma 2 Operation Arrowhead Beta (Obsolete)),(Arma 2 Operation Arrowhead),0.020173,0.020173,0.020173,1.0,49.572000,1.0,0.019766,inf,1.0,1.0,1.0,1.0
2,(Call of Duty Black Ops),(Call of Duty Black Ops - Multiplayer),0.020495,0.020495,0.020495,1.0,48.791339,1.0,0.020075,inf,1.0,1.0,1.0,1.0
3,(Call of Duty Black Ops - Multiplayer),(Call of Duty Black Ops),0.020495,0.020495,0.020495,1.0,48.791339,1.0,0.020075,inf,1.0,1.0,1.0,1.0
4,(Call of Duty Modern Warfare 2),(Call of Duty Modern Warfare 2 - Multiplayer),0.027677,0.027677,0.027677,1.0,36.131195,1.0,0.026911,inf,1.0,1.0,1.0,1.0


---

## 6. Génération des règles au format CLIPS

Pour utiliser CLIPS, nous convertissons les noms de jeux en symboles compatibles et écrivons les règles simples (1 antécédent → 1 conséquent) dans un fichier `.clp`. Cette fonction permet de générer automatiquement le fichier.

In [13]:
def sanitize(name: str) -> str:
    # Remplace les caractères spéciaux par des underscores
    name = re.sub(r"[&()/\-]", " ", name)
    name = re.sub(r"[^a-zA-Z0-9]", "_", name)
    name = re.sub(r"_+", "_", name).strip("_")
    return name


def generate_clips_rules(rules: pd.DataFrame,
                          output_file: str = "regles_steam.clp",
                          max_antecedents: int = 1,
                          max_consequents: int = 1) -> None:

    with open(output_file, "w", encoding="utf-8") as f:
        for i, row in rules.iterrows():
            antecedents = list(row['antecedents'])
            consequents = list(row['consequents'])

            # Filtre pour règles simples
            if len(antecedents) != max_antecedents or len(consequents) != max_consequents:
                continue

            ant = sanitize(antecedents[0])
            cons = sanitize(consequents[0])
            f.write(f"(defrule regle-{i+1}\n")
            f.write(f"  (achat {ant})\n")
            f.write("  =>\n")
            f.write(f"  (assert (achat {cons}))\n")
            f.write(f"  (printout t \"Regle activée : {ant} => {cons}\" crlf))\n\n")

    print(f"Fichier CLIPS généré : {output_file}")

# Génération du fichier de règles
generate_clips_rules(rules)

Fichier CLIPS généré : regles_steam.clp


---

## 7. Test interactif des règles CLIPS

Nous chargeons le fichier `.clp` dans l'environnement CLIPS, puis proposons un test interactif permettant à l'utilisateur d'entrer un jeu et d'observer les recommandations générées.

Entrez un jeu pour tester les règles (ou laissez vide pour quitter)

Voici quelques idée de test :
> Portal
>
> Half Life
>
> Robocraft


In [16]:
# Initialisation de l'environnement CLIPS
environment = clips.Environment()
environment.load("regles_steam.clp")

# Fonction de test interactif

def clips_test(jeu_test: str = "Portal") -> None:
    
    environment.assert_string(f"(achat {sanitize(jeu_test)})")

    for fact in environment.facts():
        print("Fait initial :", fact)

    environment.run()
    print("\n")

    # Afficher les faits après exécution
    for fact in environment.facts():
        print("Fait après exécution :", fact)

# Boucle de test
while True:
    print("\n--- Test interactif des règles CLIPS ---")
    environment.reset()
    jeu_test = input("Entrez le nom du jeu à tester (ex : Portal): ")
    if jeu_test :
        clips_test(jeu_test)
    else:
        print("Fin du test interactif.")
        break


--- Test interactif des règles CLIPS ---
Fait initial : (achat Portal)


Fait après exécution : (achat Portal)
Fait après exécution : (achat Half_Life_2_Lost_Coast)
Fait après exécution : (achat Counter_Strike_Source)
Fait après exécution : (achat Half_Life_2_Deathmatch)
Fait après exécution : (achat Half_Life_2)

--- Test interactif des règles CLIPS ---
Fait initial : (achat Portal_2)


Fait après exécution : (achat Portal_2)
Fait après exécution : (achat Portal)
Fait après exécution : (achat Half_Life_2_Lost_Coast)
Fait après exécution : (achat Counter_Strike_Source)
Fait après exécution : (achat Half_Life_2_Deathmatch)
Fait après exécution : (achat Half_Life_2)

--- Test interactif des règles CLIPS ---
Fait initial : (achat Half_Life)


Fait après exécution : (achat Half_Life)
Fait après exécution : (achat Counter_Strike)
Fait après exécution : (achat Counter_Strike_Condition_Zero)
Fait après exécution : (achat Counter_Strike_Condition_Zero_Deleted_Scenes)
Fait après exécution : (a

## Conclusion

Nous avons réaliser un système capable de faire des recommandations en fonction d'un jeu que vous appréciez. Bien sur le système n'est pas comparable à ce que propose la plateforme Steam mais il permet de comprendre comment des système de recommandation peuvent fonctionner.

Malgrès les 200 000 entrées le programme reste rapide à executer (quelques secondes).

## Critique et amélioration

Le système actuelle pourrait etre ameliorer de deux manières :

1. Utiliser un système de tag pour affiner les associations.

2. Augmenter considérablement le nombre de donnée.